## FINK Lightcurve Plotting

The objective of this notebook is to demonstrate plotting optical ligthcurves for the crossmatched sources identified by FINK. Here, I can edit things in isolation of a larger notebook that contains other bits of code unrelated to optical light curve plotting. The code and relevant imports are also contained in the Projecttools.py file.

In [ ]:
#here are the necessary imports
import os
import sys
import gc
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from io import StringIO
from vasttools.pipeline import Pipeline
from vasttools.query import Query
import Projecttools as pro #brand new module for frequently used code!

%matplotlib inline

In [ ]:
#pulling out crossmatch catalogue and sorting it into families
cms = pd.read_pickle('Fink_2020_sources_matched_to_VAST_all_sources.pickle')

pro.family_sort(cms)
cms.groupby('family').size().sort_values(ascending=False)

So what I want to do here is load in the FINK data for the catalogue sources, then select one of those sources to have their light curve plotted. You can make a request for source info in the FINK_Query Notebook and load it into here:

In [ ]:
fsd = pd.read_csv('/home/jovyan/work/Project_VAST_FINK/FINK_Source_Data/FSD_1.csv')

In [ ]:
#we can then select the rows in cms that have the same id as the loaded fsd file:
cms_group = cms[cms['objectId'].isin(fsd['i:objectId'])]

In [ ]:
#then we can take out a further subset of those loaded sources based on family
my_sources=cms_group.query('family == "AGN"')
my_sources

In [ ]:
#defining the photometry plotting function here to save having to rerun it everytime
#This function is also available in Projecttools.py:

def plot_photometry(pdf):
    fig = plt.figure(figsize=(15, 6))

    colordic = {1: 'C0', 2: 'C1'}

    for filt in np.unique(pdf['i:fid']):
        maskFilt = pdf['i:fid'] == filt

        # The column `d:tag` is used to check data type
        maskValid = pdf['d:tag'] == 'valid'
        plt.errorbar(
            pdf[maskValid & maskFilt]['i:jd'].apply(lambda x: x - 2400000.5),
            pdf[maskValid & maskFilt]['i:magpsf'],
            pdf[maskValid & maskFilt]['i:sigmapsf'],
            ls = '', marker='o', color=colordic[filt]
        )

        maskUpper = pdf['d:tag'] == 'upperlim'
        plt.plot(
            pdf[maskUpper & maskFilt]['i:jd'].apply(lambda x: x - 2400000.5),
            pdf[maskUpper & maskFilt]['i:diffmaglim'],
            ls='', marker='^', color=colordic[filt], markerfacecolor='none'
        )

        maskBadquality = pdf['d:tag'] == 'badquality'
        plt.errorbar(
            pdf[maskBadquality & maskFilt]['i:jd'].apply(lambda x: x - 2400000.5),
            pdf[maskBadquality & maskFilt]['i:magpsf'],
            pdf[maskBadquality & maskFilt]['i:sigmapsf'],
            ls='', marker='v', color=colordic[filt]
        )

    plt.gca().invert_yaxis()
    plt.xlabel('Modified Julian Date')
    plt.ylabel('Magnitude')
    plt.show()

Now, we parse out the specific source we want to plot, which is going to involve selecting the multiple rows
attributed to a specific source within fsd, preserving it as a DataFrame, then feeding it into the 'plot_photometry' function.

In [ ]:
#This is fairly simple, we just specify a FINK ID from our cms_group list:
ID='ZTF18acetvcr'

#then define a new variable 'fsd_source, which is just all the rows in fsd such that the key 'i:objectId' is our ID of interest:
fsd_source=fsd[fsd['i:objectId'] == ID]

In [ ]:
plot_photometry(fsd_source)

# FINK Cutouts

In this secton, We retrieve the PNG cutouts for the source whose lighturve I've just plotted:

NOTE: At time of writing, the FSD.csv files that contain cutout information is saved as a string, and as such cant be read by the below functions. You'll need to figure out how to how to turn it back into an array of float64 numbers, or alternatively query and save the appropriate columns as a separate .pkl file to be loaded in.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 6))

for col in cutouts:
    data = fsd_source[col].values[0]
    axes[cutouts.index(col)].imshow(np.arcsinh(data))